In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os

os.environ["CUDA_VISIBLE_DEVICES"] = '-1'
np.set_printoptions(linewidth=110, precision=4)

from tensorflow.keras.layers import Input, Conv2D, Activation
from tensorflow.keras.models import Model

### SparseConv2D

see [sparseconvolution.ipynb](sparseconvolution.ipynb)

### MaxPoolingWithArgmax2D and MaxUnpooling2D

In [ ]:
from layers import MaxPoolingWithArgmax2D, MaxUnpooling2D

size = 2
x = x_in = Input(shape=(32, 32, 10))
x, m = MaxPoolingWithArgmax2D(pool_size=size)(x)
x_p = x
x = MaxUnpooling2D(size=size)([x,m])
model = Model(x_in, [x,m,x_p])

print(model.input_shape)
print(model.output_shape)

batch = np.random.rand(4,32,32,10).astype('float32')

preds = model.predict(batch)
print([e.shape for e in preds])

for i in [0]:
    print()
    print(batch[0][:size,:size,i])
    print(preds[2][0][0,0,i], int(preds[1][0][0,0,i]))
    print(preds[0][0][:size,:size,i])
    
plt.figure(figsize=(16,4))
plt.subplot(141); plt.title('input')
plt.imshow(batch[0,:,:,0])
plt.subplot(142); plt.title('maxpooling')
plt.imshow(preds[2][0,:,:,0])
plt.subplot(143); plt.title('indices')
plt.imshow(preds[1][0,:,:,0])
plt.subplot(144); plt.title('unpooling')
plt.imshow(preds[0][0,:,:,0])
plt.show()

### AddCoord2D

In [ ]:
from layers import AddCoords2D

x = x_in = Input(shape=(28, 28, 3))
x = AddCoords2D()(x)
model = Model(x_in, x)

print(model.input_shape)
print(model.output_shape)

batch = np.random.rand(4,28,28,3).astype('float32')

preds = model.predict(batch)
print(preds[0][-2,:,:])

### DepthwiseConv2D

In [ ]:
from layers import DepthwiseConv2D

cm = 2 # channel_multiplier

x = x_in = Input(shape=(28, 28, 3))
x = Conv2D(16, (3,3), padding='same')(x)
x = DepthwiseConv2D(cm, (3,3), padding='same')(x)
x = Conv2D(16*cm, (1,1), padding='same')(x)
model = Model(x_in, x)

print(model.input_shape)
print(model.output_shape)

batch = np.random.rand(4,28,28,3).astype('float32')

preds = model.predict(batch)
print(preds[0][0,0,:])

### LayerNormalization

In [ ]:
from layers import LayerNormalization

x = x_in = Input(shape=(28, 28, 3))
x = LayerNormalization()(x)
model = Model(x_in, x)

print(model.input_shape)
print(model.output_shape)

batch = np.random.rand(4,28,28,3).astype('float32')*100

preds = model.predict(batch)

print(batch[0,:,0,0])
print(preds[0,:,0,0])

### Conv2DWeightNorm

In [ ]:
# TODO

from layers import Conv2DWeightNorm

x = x_in = Input(shape=(28, 28, 3))
x1 = Conv2D(10, 3)(x)
x2 = Conv2DWeightNorm(10, 3)(x)
model = Model([x_in], [x1, x2])

batch = np.random.rand(4,28,28,3).astype('float32')*100
preds = model.predict(batch)

print(model.input_shape)
print(model.output_shape)

batch = np.random.rand(4,28,28,3).astype('float32')*100

preds = model.predict(batch)

print(batch[0,:,0,0])
print(preds[0][0,:,0,0])
print(preds[1][0,:,0,0])

### Resize2D

In [ ]:
from layers import Resize2D

s = 12
s1, s2 = s//2, s*2

x = x_in = Input(shape=(s, s, 1))
x1 = Resize2D((s1,s1), 'bilinear')(x)
x2 = Resize2D((s2,s2), 'bilinear')(x)
x3 = Resize2D((s1,s1), 'nearest')(x)
x4 = Resize2D((s2,s2), 'nearest')(x)
model = Model(x_in, [x1,x2,x3,x4])

print(model.input_shape)
print(model.output_shape)

batch = np.random.rand(4,s,s,1).astype('float32')

preds = model.predict(batch)

plt.figure(figsize=(16,4))
plt.subplot(151); plt.title('input')
plt.imshow(batch[0,:,:,0])
plt.subplot(152); plt.title('bilinear %.1f' % (s1/s,))
plt.imshow(preds[0][0,:,:,0])
plt.subplot(153); plt.title('bilinear %.1f' % (s2/s,))
plt.imshow(preds[1][0,:,:,0])
plt.subplot(154); plt.title('nearest %.1f' % (s1/s,))
plt.imshow(preds[2][0,:,:,0])
plt.subplot(155); plt.title('nearest %.1f' % (s2/s,))
plt.imshow(preds[3][0,:,:,0])
plt.show()